In [90]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from imblearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, StackingClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, mean_squared_error, balanced_accuracy_score



In [31]:
df = pd.read_csv('../../2-cleaning-EDA/data/pros.csv')

In [32]:
df.shape

(958, 48)

In [33]:
df.drop(columns = 'Unnamed: 0', inplace = True)

In [34]:
X = df.drop(columns = 'player_name')
y = df.player_name

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [35]:
y_train.value_counts(normalize = True)

Vati           0.398329
Oski           0.337047
M0nkey M00n    0.264624
Name: player_name, dtype: float64

In [40]:
ss = StandardScaler()

X_train = pd.DataFrame(ss.fit_transform(X_train), columns = ss.get_feature_names_out())
X_test = pd.DataFrame(ss.transform(X_test), columns = ss.get_feature_names_out())


In [41]:
knn = KNeighborsClassifier()

knn.fit(X_train_sc, y_train)

KNeighborsClassifier()

In [42]:
knn.score(X_train_sc, y_train)

0.8857938718662952

In [43]:
knn.score(X_test_sc, y_test)

0.825

In [87]:
et = ExtraTreesClassifier()

et.fit(X_train, y_train)


ExtraTreesClassifier()

In [88]:
et.score(X_train, y_train)

1.0

In [89]:
et.score(X_test, y_test)

0.8916666666666667

In [93]:
poly = PolynomialFeatures(interaction_only=True)

X_train_poly = pd.DataFrame(poly.fit_transform(X_train), columns=poly.get_feature_names_out())
X_test_poly = pd.DataFrame(poly.transform(X_test), columns=poly.get_feature_names_out())

In [101]:
X_train_poly.shape

(718, 1082)

In [94]:
knn.fit(X_train_poly, y_train)

KNeighborsClassifier()

In [95]:
knn.score(X_train_poly, y_train)

0.8161559888579387

In [96]:
knn.score(X_test_poly, y_test)

0.75

In [97]:
et.fit(X_train_poly, y_train)

ExtraTreesClassifier()

In [100]:
et.score(X_train_poly, y_train)

1.0

In [98]:
et.score(X_test_poly, y_test)

0.8875

In [ ]:
def get_accuracies(X_train, y_train, X_test, y_test, model, model_name:str, scores_df=None, norm=None):
    
    # Takes training and test splits, a desired model and model name (string),
    # current scoring dataframe and norm parameter for the heatmap output
    
    # Creates columns for scoring metrics on first run
    if scores_df is None:
        scores_df = pd.DataFrame(columns = ['train_acc', 'test_acc', 'bal_acc'])
    
    # fits model to training data
    model.fit(X_train, y_train)
    
    # produces a confusion matrix for the prediction and true values of the test data
    ConfusionMatrixDisplay.from_estimator(model, X_test, y_test, cmap='Blues', normalize=norm)
    
    # predicted subreddits for each row, used to calculate balanced accuracy
    preds = model.predict(X_test)
    train_acc = model.score(X_train, y_train)
    test_acc = model.score(X_test, y_test)
    bal_acc = balanced_accuracy_score(y_test, preds)

    # dataframe updated with model name and corresponding scores
    scores_df.loc[model_name,:] = [train_acc, test_acc, bal_acc]
    
    print(scores_df)
    
    return scores_df